# Energy Spectrum Comparison

RAP energy spectra with penalty terms:
- (a) Repetition code [[3,1,3]] with stabilizer penalty
- (b) Bacon-Shor [[4,1,1,2]] with gauge penalty

In [ ]:
import sys, os
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

sys.path.insert(0, os.path.dirname(os.getcwd()))
from qec_config import QECConfig, BaconShorConfig, PlotConfig

PlotConfig.apply()

# Parameters
EP_MHZ = 75
N_POINTS = 1001
OUTPUT_DIR = Path('../figs/main_figure')
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Initialize
rep = QECConfig(n_points=N_POINTS)
bs = BaconShorConfig(n_points=N_POINTS)
Ep = rep.Ep_MHz_to_rad(EP_MHZ)

# Load/compute spectra
rep_E, rep_idx = rep.get_or_compute_spectrum(Ep=Ep)
bs_E, bs_idx = bs.get_or_compute_spectrum(Ep=Ep)

In [ ]:
# Plotting helper - use known correct ordering for repetition code
REP_CODE_LABELS = {
    # From lowest to highest energy at end of protocol
    # Order: |1_L⟩, |0_L⟩, |011⟩, |110⟩, |100⟩, |001⟩, |101⟩, |010⟩
    'error_labels_sorted': ['011', '110', '100', '001', '101', '010']
}

def plot_spectrum(ax, config, energies, idx_series, title, Ep, show_ylabel=True, show_labels=True, is_bacon_shor=False):
    """Plot full energy spectrum with state labels."""
    t = config.t_list * config.TO_TIME_UNITS
    C0, C1 = PlotConfig.COLORS_LOGICAL
    C_ERR = PlotConfig.COLOR_ERROR_DEFAULT
    
    # Logical states
    E0 = config.break_at_swaps(energies[0] * config.TO_FREQ_UNITS, idx_series[0])
    E1 = config.break_at_swaps(energies[1] * config.TO_FREQ_UNITS, idx_series[1])
    ax.plot(t, E0, color=C0, lw=5, zorder=10)
    ax.plot(t, E1, color=C1, lw=5, zorder=10)
    
    # Error states
    for i in range(2, config.dim):
        ax.plot(t, energies[i] * config.TO_FREQ_UNITS, 
                color=C_ERR, lw=2.5, ls='--', alpha=0.7, zorder=5)
    
    # Set x-axis limits first - leave space for labels on right
    t_end = t[-1]
    if is_bacon_shor:
        ax.set_xlim([0, t_end + 1.6])  # More space for longer labels
        x_label_pos = t_end + 0.06
    else:
        ax.set_xlim([0, t_end + 0.8])
        x_label_pos = t_end + 0.06
    
    # Right-side labels (now inside plot area)
    if show_labels:
        fontsize_label = 20
        fontsize_err = 18
        
        # Logical state labels - different for Bacon-Shor
        if is_bacon_shor:
            label_0L = r'$|0_L\rangle \otimes |0_G\rangle$'
            label_1L = r'$|1_L\rangle \otimes |0_G\rangle$'
        else:
            label_0L = r'$|0_L\rangle$'
            label_1L = r'$|1_L\rangle$'
        
        ax.text(x_label_pos, energies[0][-1] * config.TO_FREQ_UNITS, 
                label_0L, fontsize=fontsize_label, va='center', color=C0, fontweight='bold')
        ax.text(x_label_pos, energies[1][-1] * config.TO_FREQ_UNITS, 
                label_1L, fontsize=fontsize_label, va='center', color=C1, fontweight='bold')
        
        # Error state labels
        err_finals = [(energies[i][-1] * config.TO_FREQ_UNITS, i) for i in range(2, config.dim)]
        err_finals.sort(key=lambda x: x[0])  # Sort by energy (low to high)
        
        if config.dim <= 8:  # Few states - label each with known ordering
            labels = REP_CODE_LABELS['error_labels_sorted']
            
            # Custom positioning: space |011⟩,|110⟩ lower away from |100⟩,|001⟩
            min_sep = 18
            pos = []
            for i, (e, _) in enumerate(err_finals):
                if i == 0:
                    pos.append(e)
                elif i == 2:  # |100⟩ - add extra gap after |110⟩
                    pos.append(max(e, pos[-1] + min_sep + 10))
                else:
                    pos.append(max(e, pos[-1] + min_sep))
            
            for i, ((e, idx), p) in enumerate(zip(err_finals, pos)):
                label = labels[i] if i < len(labels) else '?'
                ax.text(x_label_pos, p, f'$|{label}\\rangle$', fontsize=fontsize_err, 
                        va='center', color=C_ERR, alpha=0.85)
        else:  # Many states - group label
            mid = (err_finals[0][0] + err_finals[-1][0]) / 2
            ax.text(x_label_pos, mid, 'Error\nstates', fontsize=fontsize_err, 
                    va='center', color=C_ERR, alpha=0.85)
    
    # Styling - no individual x label
    if show_ylabel:
        ax.set_ylabel('Energy [MHz]', fontsize=24)
    ax.set_title(title, fontsize=24)
    ax.tick_params(labelsize=20)
    ax.grid(True, alpha=0.3, ls='--', lw=1.5)
    
    # Reduce number of ticks
    ax.xaxis.set_major_locator(plt.MaxNLocator(3))
    ax.yaxis.set_major_locator(plt.MaxNLocator(5))

# Create figure with shared y-axis
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 8), sharey=True)
Ep = rep.Ep_MHz_to_rad(EP_MHZ)

plot_spectrum(ax1, rep, rep_E, rep_idx, r'(a) Repetition $[[3,1,3]]$', Ep, show_ylabel=True, show_labels=True, is_bacon_shor=False)
plot_spectrum(ax2, bs, bs_E, bs_idx, r'(b) Bacon-Shor $[[4,1,1,2]]$', Ep, show_ylabel=False, show_labels=True, is_bacon_shor=True)

# Adjust y-axis padding after both plots
y_min, y_max = ax1.get_ylim()
y_range = y_max - y_min
ax1.set_ylim([y_min - 0.1 * y_range, y_max + 0.1 * y_range])

plt.tight_layout()
plt.subplots_adjust(wspace=0.05, bottom=0.12)

# Add shared x label
fig.text(0.5, 0.02, r'Time [$\mu$s]', ha='center', fontsize=24)

plt.show()

In [3]:
# Save figure
fig.savefig(OUTPUT_DIR / 'energy_spectrum_comparison.pdf', bbox_inches='tight')
fig.savefig(OUTPUT_DIR / 'energy_spectrum_comparison.svg', bbox_inches='tight')
fig.savefig(OUTPUT_DIR / 'energy_spectrum_comparison.png', dpi=300, bbox_inches='tight')

# Save data
data = {'Ep_MHz': EP_MHZ, 't_rep_us': rep.t_list * rep.TO_TIME_UNITS, 
        't_bs_us': bs.t_list * bs.TO_TIME_UNITS}
for i in range(rep.dim):
    data[f'rep_E{i}_MHz'] = rep_E[i] * rep.TO_FREQ_UNITS
for i in range(bs.dim):
    data[f'bs_E{i}_MHz'] = bs_E[i] * bs.TO_FREQ_UNITS
data['rep_idx_0L'], data['rep_idx_1L'] = np.array(rep_idx[0]), np.array(rep_idx[1])
data['bs_idx_0L'], data['bs_idx_1L'] = np.array(bs_idx[0]), np.array(bs_idx[1])

np.savez_compressed(OUTPUT_DIR / 'plot_data.npz', **data)
print(f"Saved to {OUTPUT_DIR}")

'created' timestamp seems very low; regarding as unix timestamp
'modified' timestamp seems very low; regarding as unix timestamp
'created' timestamp seems very low; regarding as unix timestamp
'modified' timestamp seems very low; regarding as unix timestamp
'created' timestamp seems very low; regarding as unix timestamp
'modified' timestamp seems very low; regarding as unix timestamp
'created' timestamp seems very low; regarding as unix timestamp
'modified' timestamp seems very low; regarding as unix timestamp


Saved to ../figs/main_figure
